In [1]:
import os
import json
import json5
from time import sleep
from openai import OpenAI
from tqdm import tqdm

In [2]:
client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=os.getenv("OPENAI_API_KEY"))

def safe_json_parse(content):
    try:
        return json5.loads(content)
    except Exception as e:
        print("⚠️ Malformed JSON:", e)
        return None

def parse_resume_with_llm(text):
    prompt = f"""
    You are a CV parsing system.
    Extract the following fields from the CV below and return STRICTLY valid JSON:
    - name
    - email
    - phone
    - location
    - summary
    - skills (list)
    - experience (list of objects: title, company, start_date, end_date, description)
    - education (list of objects: degree, school, year)
    - certifications (list)
    - languages (list)

    CV:
    --------------
    {text}
    --------------
    
    Reply ONLY with valid JSON.
    """
    response = client.chat.completions.create(
        model="openai/gpt-5-nano",
        messages=[{"role": "user", "content": prompt}]
    )
    return safe_json_parse(response.choices[0].message.content)

def process_resumes(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    for filename in tqdm([f for f in os.listdir(input_folder) if f.endswith(".txt")],
                         desc="Processing CVs", unit="CV"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".json")
        
        with open(input_path, "r", encoding="utf-8", errors="ignore") as f:
            text = f.read()
        
        parsed = None
        for _ in range(10):
            parsed = parse_resume_with_llm(text)
            if parsed:
                break
            sleep(1)
        
        if not parsed:
            print(f"⛔ Failed to parse {filename}")
            continue
        
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(parsed, f, indent=4, ensure_ascii=False)

In [ ]:
process_resumes("./data/resume_extract", "./data/resume_extract_json")

Processing CVs:   2%|▏         | 48/2549 [44:40<39:27:09, 56.79s/CV]